In [1]:
# import json
import numpy as np
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage.filters import threshold_otsu
from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.measure import label, regionprops
import json
import os
import uuid
from PIL import Image
import random
import math
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import glob

In [2]:
def reset(percent=20):
    return random.randrange(100) < percent

In [3]:
# Training and Validate Sets
degs = [45,90,135,180,225,270,315]
img_bbs = []
count = 1
phase_count = {
    "train": 1,
    "test": 1
}
phase_count["train"] = 1
phase_count["test"] = 1

labels = os.path.join('training.json')
with open(labels,'r') as json_file:
    datas = json.load(json_file)
    for data in datas:
        read_path = 'tmp_images' + data['image']['pathname']
        print(read_path)
        phase = "train" if (count % 2) == 0 else "test"            
        img_name_new = "pv-" + phase + "-{:010d}".format(phase_count[phase])
        print(img_name_new)
        save_path = os.path.join('images',img_name_new+".jpg")
        img = Image.open(read_path)
        objects = data['objects']
        rects = []
        result = ""
        img = np.array(img)
        h,w,c = img.shape
        RBCs_flag = reset(17)
        for obj in objects:
            minr = obj['bounding_box']['minimum']['r']
            minc = obj['bounding_box']['minimum']['c']
            maxr = obj['bounding_box']['maximum']['r']
            maxc = obj['bounding_box']['maximum']['c']
            x1 = minc / w
            y1 = minr / h
            bw = (maxc - minc) / w
            bh = (maxr - minr) / h
            xc = x1 + (bw / 2)
            yc = y1 + (bh / 2)
            category = obj['category']
            if (category == "red blood cell") and (RBCs_flag or phase == "test"):
                result += "\n"+ "0 " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            elif category == "ring" or category == "trophozoite" or category == "schizont" or category == "gametocyte":
                result += "\n" + "1 " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                rects.append(BoundingBox(x1=minc,y1=minr,x2=maxc,y2=maxr,label=1))
        result = result[1:]
        if len(result) == 0:
            count += 1
            phase_count[phase] += 1
            continue
        Image.fromarray(img).save(save_path)
        f = open(os.path.join('labels',img_name_new+".txt"),'w')
        f.write(result)
        f.close()
        bbs = BoundingBoxesOnImage(rects,shape=img.shape)
        count += 1
        phase_count[phase] += 1
        if phase == "train":
            for deg in degs:
                image_aug, bbs_aug = iaa.Affine(rotate=deg,mode="reflect")(image=img, bounding_boxes=bbs)
                clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=True).clip_out_of_image()
                h,w,c = image_aug.shape
                result = ""
                img_name_new = "pv-" + phase + "-{:010d}".format(phase_count[phase])
                for bb in clip_off_bbs.bounding_boxes:
                    minx = int(bb.x1)
                    miny = int(bb.y1)
                    maxx = int(bb.x2)
                    maxy = int(bb.y2)
                    bb_w = (maxx-minx)/w
                    bb_h = (maxy-miny)/h
                    bb_xc = (minx + ((maxx-minx)/2))/w
                    bb_yc = (miny + ((maxy-miny)/2))/h
                    result += "\n" + str(bb.label)+" "+str(bb_xc)+" "+str(bb_yc)+ " "+str(bb_w)+ " "+str(bb_h)
                if len(clip_off_bbs.bounding_boxes) > 0:
                    print("{} {}".format(phase_count[phase],deg))
                    print(img_name_new)
                    result = result[1:]
                    save_path = os.path.join('images',img_name_new+".jpg")
                    Image.fromarray(image_aug).save(save_path)
                    f = open(os.path.join('labels',img_name_new+".txt"),'w')
                    f.write(result)
                    f.close()
                    phase_count[phase] += 1

tmp_images/images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png
pv-test-0000000001
tmp_images/images/10be6380-cbbb-4886-8b9e-ff56b1710576.png
pv-train-0000000001
tmp_images/images/6b14c855-8561-417c-97a4-63fa552842fd.png
pv-test-0000000002
tmp_images/images/13099edb-35d9-438f-b093-2cf2ebf9d255.png
pv-train-0000000002
3 90
pv-train-0000000003
4 135
pv-train-0000000004
5 180
pv-train-0000000005
6 270
pv-train-0000000006
7 315
pv-train-0000000007
tmp_images/images/2559636b-f01a-4414-93da-210c3b12d153.png
pv-test-0000000003
tmp_images/images/0dcca702-a4ef-4fb3-a940-9c0c140b21c7.png
pv-train-0000000008
9 45
pv-train-0000000009
10 90
pv-train-0000000010
11 135
pv-train-0000000011
12 180
pv-train-0000000012
13 225
pv-train-0000000013
14 270
pv-train-0000000014
15 315
pv-train-0000000015
tmp_images/images/7c66d2fd-68ee-4628-b860-9d7ad25b2a80.png
pv-test-0000000004
tmp_images/images/84c88143-940d-4b62-bbd2-7acc4cadcb6b.png
pv-train-0000000016
tmp_images/images/c6d97845-8ad2-4ab7-b385-66613628df5b.pn

144 45
pv-train-0000000144
145 90
pv-train-0000000145
146 135
pv-train-0000000146
147 180
pv-train-0000000147
148 225
pv-train-0000000148
149 270
pv-train-0000000149
150 315
pv-train-0000000150
tmp_images/images/0f26699f-a776-4424-b092-27da9a1d12e2.png
pv-test-0000000035
tmp_images/images/98efca97-37d2-4459-aca6-375ecc0c4539.png
pv-train-0000000151
152 45
pv-train-0000000152
153 135
pv-train-0000000153
154 180
pv-train-0000000154
155 225
pv-train-0000000155
156 315
pv-train-0000000156
tmp_images/images/f1de7c7e-e7d3-45ae-a66c-58538bf6cac1.png
pv-test-0000000036
tmp_images/images/9615ff33-b3fc-4d1e-9c05-0b8e3ef936df.png
pv-train-0000000157
tmp_images/images/a2d07312-2f97-40d2-8b09-7f2aca7b16fb.png
pv-test-0000000037
tmp_images/images/b91b1641-ee06-44d8-94cc-24cdb583cc37.png
pv-train-0000000158
159 45
pv-train-0000000159
160 90
pv-train-0000000160
161 135
pv-train-0000000161
162 180
pv-train-0000000162
163 225
pv-train-0000000163
164 270
pv-train-0000000164
165 315
pv-train-0000000165
tm

311 225
pv-train-0000000311
312 270
pv-train-0000000312
313 315
pv-train-0000000313
tmp_images/images/1493d140-c9d5-4c3a-9caf-3eafb290a49b.png
pv-test-0000000063
tmp_images/images/98c25ffe-c329-420a-bc22-b082f565eac9.png
pv-train-0000000314
tmp_images/images/9af4b8ae-c912-41fd-b4c0-34864c75360f.png
pv-test-0000000064
tmp_images/images/933e7e07-fc3d-42eb-be0c-6b1623ee60fa.png
pv-train-0000000315
tmp_images/images/694222bb-cddf-469a-9526-f873c6fba58e.png
pv-test-0000000065
tmp_images/images/6bb4892c-27a9-4642-aba1-4b73f38f757c.png
pv-train-0000000316
317 45
pv-train-0000000317
318 135
pv-train-0000000318
319 180
pv-train-0000000319
320 225
pv-train-0000000320
321 315
pv-train-0000000321
tmp_images/images/65e3d6a6-012c-4b40-9a42-83078eb85e31.png
pv-test-0000000066
tmp_images/images/9a1cd456-310c-47b6-a1f8-0fbd8e33d1fd.png
pv-train-0000000322
tmp_images/images/ae779806-6d01-4dc6-89df-294c368db6bc.png
pv-test-0000000067
tmp_images/images/e234ce8b-4ec4-4d93-9068-d7854d77ddf4.png
pv-train-000

tmp_images/images/f8a8b148-60e4-4a5c-ae19-941b19ec0846.png
pv-train-0000000464
465 45
pv-train-0000000465
466 90
pv-train-0000000466
467 135
pv-train-0000000467
468 180
pv-train-0000000468
469 225
pv-train-0000000469
470 270
pv-train-0000000470
471 315
pv-train-0000000471
tmp_images/images/ca16a5ad-b106-4e84-8a43-178710e854ba.png
pv-test-0000000094
tmp_images/images/b234b6ea-b54d-470c-8461-ba7f4a024796.png
pv-train-0000000472
473 45
pv-train-0000000473
474 90
pv-train-0000000474
475 135
pv-train-0000000475
476 180
pv-train-0000000476
477 225
pv-train-0000000477
478 270
pv-train-0000000478
479 315
pv-train-0000000479
tmp_images/images/984caf56-f400-4a29-be97-a4d6b9d1468d.png
pv-test-0000000095
tmp_images/images/87d6343b-70dd-4cdd-939f-d3ac0ca2e700.png
pv-train-0000000480
tmp_images/images/313ed8d9-3c00-4415-9de0-0eda07c60ad9.png
pv-test-0000000096
tmp_images/images/5a0628af-4c0a-4ba6-a935-4ec94c9c700e.png
pv-train-0000000481
tmp_images/images/3b55c42f-ad59-422f-ba99-4f4f4ca9cb2e.png
pv-

tmp_images/images/c7c1b873-0548-4a73-9618-f60912e164df.png
pv-train-0000000627
628 45
pv-train-0000000628
629 90
pv-train-0000000629
630 135
pv-train-0000000630
631 180
pv-train-0000000631
632 225
pv-train-0000000632
633 270
pv-train-0000000633
634 315
pv-train-0000000634
tmp_images/images/7e25ced3-5caf-47f8-ad0c-42946bd4468b.png
pv-test-0000000123
tmp_images/images/66557580-fd38-4d4b-80d8-88611ecc78c6.png
pv-train-0000000635
636 135
pv-train-0000000636
637 180
pv-train-0000000637
638 315
pv-train-0000000638
tmp_images/images/1e978dfa-d339-4025-809d-09cee76728ed.png
pv-test-0000000124
tmp_images/images/acdc6e12-73d9-44ba-be8c-2a56aaffef4f.png
pv-train-0000000639
640 45
pv-train-0000000640
641 90
pv-train-0000000641
642 135
pv-train-0000000642
643 180
pv-train-0000000643
644 225
pv-train-0000000644
645 270
pv-train-0000000645
646 315
pv-train-0000000646
tmp_images/images/d5009906-bce1-4353-90b4-f4c79362f4cf.png
pv-test-0000000125
tmp_images/images/69569ecf-cfa1-4ddd-9342-b22c23387fdb.pn

...

2705 225
pv-train-0000002705
2706 270
pv-train-0000002706
2707 315
pv-train-0000002707
tmp_images/images/6dfafdf1-f588-4e80-8cf8-0126e1bf7af9.png
pv-test-0000000496
tmp_images/images/76021d69-8919-4688-95ef-7418eff620cc.png
pv-train-0000002708
2709 45
pv-train-0000002709
2710 90
pv-train-0000002710
2711 135
pv-train-0000002711
2712 180
pv-train-0000002712
2713 225
pv-train-0000002713
2714 270
pv-train-0000002714
2715 315
pv-train-0000002715
tmp_images/images/a74b6d6a-7e04-40ac-9292-d62f13f7ded3.png
pv-test-0000000497
tmp_images/images/dec61ae1-5037-4c3a-8952-15371becca7d.png
pv-train-0000002716
2717 45
pv-train-0000002717
2718 90
pv-train-0000002718
2719 135
pv-train-0000002719
2720 180
pv-train-0000002720
2721 225
pv-train-0000002721
2722 270
pv-train-0000002722
2723 315
pv-train-0000002723
tmp_images/images/ff3b9d4d-373c-48ef-a75a-d23e2c2ade87.png
pv-test-0000000498
tmp_images/images/5e042380-6ef2-496f-8231-0ca550fbfa71.png
pv-train-0000002724
2725 45
pv-train-0000002725
2726 90
pv-t

tmp_images/images/e2e68cb3-0588-40bb-95b4-356f9cacffc3.png
pv-train-0000002886
tmp_images/images/29d40b60-cde1-45b5-b403-692437a77ac1.png
pv-test-0000000520
tmp_images/images/5dec94a3-e3ca-4a58-9702-88e5a83ef612.png
pv-train-0000002887
2888 45
pv-train-0000002888
2889 90
pv-train-0000002889
2890 135
pv-train-0000002890
2891 180
pv-train-0000002891
2892 225
pv-train-0000002892
2893 270
pv-train-0000002893
2894 315
pv-train-0000002894
tmp_images/images/ff1b23d0-3cef-459b-b81f-39bfcdb6526f.png
pv-test-0000000521
tmp_images/images/a6391703-0820-4913-a7cd-e843ff0d84ba.png
pv-train-0000002895
tmp_images/images/0f735cca-051c-4048-a1fa-cb36f389e12f.png
pv-test-0000000522
tmp_images/images/040112be-3c8d-49aa-93f1-522f94a592ff.png
pv-train-0000002896
2897 45
pv-train-0000002897
2898 180
pv-train-0000002898
2899 225
pv-train-0000002899
tmp_images/images/74261741-4d6e-4dba-8ad4-4fbdf919c30f.png
pv-test-0000000523
tmp_images/images/32700bc7-7a86-49d3-aded-ccaaed4ddc5c.png
pv-train-0000002900
2901 1

3059 45
pv-train-0000003059
3060 90
pv-train-0000003060
3061 135
pv-train-0000003061
3062 180
pv-train-0000003062
3063 225
pv-train-0000003063
3064 270
pv-train-0000003064
3065 315
pv-train-0000003065
tmp_images/images/093a831a-bf71-4333-bc06-942ae70cb164.png
pv-test-0000000546
tmp_images/images/61575482-91e1-4db9-83c7-687b7ddbbfc4.png
pv-train-0000003066
3067 45
pv-train-0000003067
3068 90
pv-train-0000003068
3069 135
pv-train-0000003069
3070 180
pv-train-0000003070
3071 225
pv-train-0000003071
3072 270
pv-train-0000003072
3073 315
pv-train-0000003073
tmp_images/images/6ad3efc9-4392-4cee-98a5-186564a96373.png
pv-test-0000000547
tmp_images/images/64b0e49a-0e48-447d-8872-c1dd255e7a21.png
pv-train-0000003074
3075 45
pv-train-0000003075
3076 90
pv-train-0000003076
3077 135
pv-train-0000003077
3078 180
pv-train-0000003078
3079 225
pv-train-0000003079
3080 270
pv-train-0000003080
3081 315
pv-train-0000003081
tmp_images/images/005e60b6-77b8-458c-b57c-bfe0c7e7df78.png
pv-test-0000000548
tmp_i

3236 45
pv-train-0000003236
3237 90
pv-train-0000003237
3238 135
pv-train-0000003238
3239 180
pv-train-0000003239
3240 225
pv-train-0000003240
3241 270
pv-train-0000003241
3242 315
pv-train-0000003242
tmp_images/images/fa4f8d1a-cb3a-4722-a566-6c7072f30606.png
pv-test-0000000571
tmp_images/images/c35ee701-fde2-46b5-b178-1fe3f1ea7da2.png
pv-train-0000003243
3244 45
pv-train-0000003244
3245 90
pv-train-0000003245
3246 135
pv-train-0000003246
3247 180
pv-train-0000003247
3248 225
pv-train-0000003248
3249 270
pv-train-0000003249
3250 315
pv-train-0000003250
tmp_images/images/50d7c6c9-3230-48fe-947c-7dcfd0aad710.png
pv-test-0000000572
tmp_images/images/d12c5f45-6b20-46d6-a66f-c5a4683c25b3.png
pv-train-0000003251
3252 45
pv-train-0000003252
3253 90
pv-train-0000003253
3254 135
pv-train-0000003254
3255 180
pv-train-0000003255
3256 225
pv-train-0000003256
3257 270
pv-train-0000003257
3258 315
pv-train-0000003258
tmp_images/images/f48bf825-13c4-42fc-8dd8-10cc4ae6b202.png
pv-test-0000000573
tmp_i

3412 270
pv-train-0000003412
3413 315
pv-train-0000003413
tmp_images/images/19979a60-df80-4111-bf69-81a7e9c4dd0b.png
pv-test-0000000596
tmp_images/images/4d0a1598-e4dd-45c4-8a90-dd7c780fe1da.png
pv-train-0000003414
3415 45
pv-train-0000003415
3416 90
pv-train-0000003416
3417 135
pv-train-0000003417
3418 180
pv-train-0000003418
3419 225
pv-train-0000003419
3420 270
pv-train-0000003420
3421 315
pv-train-0000003421
tmp_images/images/6144887d-a1c1-46db-91a1-f7ff5bf8c8a5.png
pv-test-0000000597
tmp_images/images/036e008f-07a1-4e92-899d-1822dc390ccb.png
pv-train-0000003422
3423 45
pv-train-0000003423
3424 90
pv-train-0000003424
3425 135
pv-train-0000003425
3426 180
pv-train-0000003426
3427 225
pv-train-0000003427
3428 270
pv-train-0000003428
3429 315
pv-train-0000003429
tmp_images/images/1c90231b-2830-42d9-884e-7301edb16933.png
pv-test-0000000598
tmp_images/images/eb0173a3-7e5e-4f53-8124-03d9d6317844.png
pv-train-0000003430
3431 45
pv-train-0000003431
3432 90
pv-train-0000003432
3433 135
pv-t

In [4]:
print(phase_count["train"])
print(phase_count["test"])

3484
605


In [5]:
# Training and Validate Sets
degs = [45,90,135,180,225,270,315]
img_bbs = []
phase_count["train"] = 3484
labels = os.path.join('test.json')
with open(labels,'r') as json_file:
    datas = json.load(json_file)
    for data in datas:
        read_path = 'tmp_images' + data['image']['pathname']
        print(read_path)
        phase = "train"
        img_name_new = "pv-" + phase + "-{:010d}".format(phase_count[phase])
        print(img_name_new)
        save_path = os.path.join('images',img_name_new+".jpg")
        img = Image.open(read_path)
        objects = data['objects']
        rects = []
        result = ""
        img = np.array(img)
        h,w,c = img.shape
        for obj in objects:
            minr = obj['bounding_box']['minimum']['r']
            minc = obj['bounding_box']['minimum']['c']
            maxr = obj['bounding_box']['maximum']['r']
            maxc = obj['bounding_box']['maximum']['c']
            x1 = minc / w
            y1 = minr / h
            bw = (maxc - minc) / w
            bh = (maxr - minr) / h
            xc = x1 + (bw / 2)
            yc = y1 + (bh / 2)
            category = obj['category']
            if (category == "red blood cell"):
                result += "\n"+ "0 " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            elif category == "ring" or category == "trophozoite" or category == "schizont" or category == "gametocyte":
                result += "\n" + "1 " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                rects.append(BoundingBox(x1=minc,y1=minr,x2=maxc,y2=maxr,label=1))
        result = result[1:]
        Image.fromarray(img).save(save_path)
        f = open(os.path.join('labels',img_name_new+".txt"),'w')
        f.write(result)
        f.close()
        bbs = BoundingBoxesOnImage(rects,shape=img.shape)
        count += 1
        phase_count[phase] += 1
        if phase == "train":
            for deg in degs:
                image_aug, bbs_aug = iaa.Affine(rotate=deg,mode="reflect")(image=img, bounding_boxes=bbs)
                clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=True).clip_out_of_image()
                h,w,c = image_aug.shape
                result = ""
                img_name_new = "pv-" + phase + "-{:010d}".format(phase_count[phase])
                
                for bb in clip_off_bbs.bounding_boxes:
                    minx = int(bb.x1)
                    miny = int(bb.y1)
                    maxx = int(bb.x2)
                    maxy = int(bb.y2)
                    bb_w = (maxx-minx)/w
                    bb_h = (maxy-miny)/h
                    bb_xc = (minx + ((maxx-minx)/2))/w
                    bb_yc = (miny + ((maxy-miny)/2))/h
                    result += "\n" + str(bb.label)+" "+str(bb_xc)+" "+str(bb_yc)+ " "+str(bb_w)+ " "+str(bb_h)
                if len(clip_off_bbs.bounding_boxes) > 0:
                    print("{} {}".format(phase_count[phase],deg))
                    print(img_name_new)
                    result = result[1:]
                    save_path = os.path.join('images',img_name_new+".jpg")
                    Image.fromarray(image_aug).save(save_path)
                    f = open(os.path.join('labels',img_name_new+".txt"),'w')
                    f.write(result)
                    f.close()
                    phase_count[phase] += 1

tmp_images/images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg
pv-train-0000003484
3485 45
pv-train-0000003485
3486 90
pv-train-0000003486
3487 135
pv-train-0000003487
3488 180
pv-train-0000003488
3489 225
pv-train-0000003489
3490 270
pv-train-0000003490
3491 315
pv-train-0000003491
tmp_images/images/64985a1e-16bb-4016-a01c-c21a5b86e572.jpg
pv-train-0000003492
3493 45
pv-train-0000003493
3494 90
pv-train-0000003494
3495 135
pv-train-0000003495
3496 180
pv-train-0000003496
3497 225
pv-train-0000003497
3498 270
pv-train-0000003498
3499 315
pv-train-0000003499
tmp_images/images/c70894fa-50e8-4eed-a5a8-159a20153a49.jpg
pv-train-0000003500
3501 45
pv-train-0000003501
3502 90
pv-train-0000003502
3503 135
pv-train-0000003503
3504 180
pv-train-0000003504
3505 225
pv-train-0000003505
3506 270
pv-train-0000003506
3507 315
pv-train-0000003507
tmp_images/images/abb72ce2-4193-4dd3-a34c-61c2ee22ccff.jpg
pv-train-0000003508
3509 45
pv-train-0000003509
3510 90
pv-train-0000003510
3511 135
pv-train-0000003

3715 45
pv-train-0000003715
3716 90
pv-train-0000003716
3717 135
pv-train-0000003717
3718 180
pv-train-0000003718
3719 225
pv-train-0000003719
3720 270
pv-train-0000003720
3721 315
pv-train-0000003721
tmp_images/images/8111cff9-4c04-448e-990b-9d5010f7be56.jpg
pv-train-0000003722
3723 45
pv-train-0000003723
3724 90
pv-train-0000003724
3725 135
pv-train-0000003725
3726 180
pv-train-0000003726
3727 225
pv-train-0000003727
3728 270
pv-train-0000003728
3729 315
pv-train-0000003729
tmp_images/images/e6a7743e-4040-40fa-b097-ca6874c469b7.jpg
pv-train-0000003730
3731 45
pv-train-0000003731
3732 90
pv-train-0000003732
3733 135
pv-train-0000003733
3734 180
pv-train-0000003734
3735 225
pv-train-0000003735
3736 270
pv-train-0000003736
3737 315
pv-train-0000003737
tmp_images/images/e02301ac-68f7-4daf-b2ba-254b917f1cca.jpg
pv-train-0000003738
3739 45
pv-train-0000003739
3740 90
pv-train-0000003740
3741 180
pv-train-0000003741
3742 225
pv-train-0000003742
3743 270
pv-train-0000003743
tmp_images/images

3948 90
pv-train-0000003948
3949 135
pv-train-0000003949
3950 180
pv-train-0000003950
3951 225
pv-train-0000003951
3952 270
pv-train-0000003952
3953 315
pv-train-0000003953
tmp_images/images/294bc238-efdb-4f88-b83b-f41fe5750106.jpg
pv-train-0000003954
3955 45
pv-train-0000003955
3956 90
pv-train-0000003956
3957 135
pv-train-0000003957
3958 180
pv-train-0000003958
3959 225
pv-train-0000003959
3960 270
pv-train-0000003960
3961 315
pv-train-0000003961
tmp_images/images/34950f4e-fe44-49ec-9d49-14bd4a8ce7c0.jpg
pv-train-0000003962
3963 45
pv-train-0000003963
3964 90
pv-train-0000003964
3965 135
pv-train-0000003965
3966 180
pv-train-0000003966
3967 225
pv-train-0000003967
3968 270
pv-train-0000003968
3969 315
pv-train-0000003969
tmp_images/images/224c80b8-6c55-48cf-b719-96a47d223dc3.jpg
pv-train-0000003970
3971 45
pv-train-0000003971
3972 90
pv-train-0000003972
3973 135
pv-train-0000003973
3974 180
pv-train-0000003974
3975 225
pv-train-0000003975
3976 270
pv-train-0000003976
3977 315
pv-trai

4184 45
pv-train-0000004184
4185 90
pv-train-0000004185
4186 135
pv-train-0000004186
4187 180
pv-train-0000004187
4188 225
pv-train-0000004188
4189 270
pv-train-0000004189
4190 315
pv-train-0000004190
tmp_images/images/408745a6-79fb-4804-9c24-e2d6673f8f2e.jpg
pv-train-0000004191
4192 45
pv-train-0000004192
4193 90
pv-train-0000004193
4194 135
pv-train-0000004194
4195 180
pv-train-0000004195
4196 225
pv-train-0000004196
4197 270
pv-train-0000004197
4198 315
pv-train-0000004198
tmp_images/images/010961af-b38c-49de-aca0-e3732d73d414.jpg
pv-train-0000004199
4200 45
pv-train-0000004200
4201 90
pv-train-0000004201
4202 135
pv-train-0000004202
4203 180
pv-train-0000004203
4204 225
pv-train-0000004204
4205 270
pv-train-0000004205
4206 315
pv-train-0000004206
tmp_images/images/47112c6b-aaf6-488d-8f3b-f9ed0cce9a95.jpg
pv-train-0000004207
4208 45
pv-train-0000004208
4209 90
pv-train-0000004209
4210 135
pv-train-0000004210
4211 180
pv-train-0000004211
4212 225
pv-train-0000004212
4213 270
pv-train

In [6]:
print(phase_count["train"])
print(phase_count["test"])

4385
605


In [7]:
# get images from pv_test (Only test)
labels = os.path.join('pv_test.json')

phase_count["test"] = 605

phase = "test"
degs = [45,90,135,180,225,270,315]

with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        leng = len(images[key]["regions"])
        bboxs = ""
        if leng > 0:
            name = "pv-" + phase + "-{:010d}".format(phase_count[phase])
            print("{} {}".format(count,url))
            print(name)
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('images',name+".jpg"))
            f = open(os.path.join('labels',name+".txt"),'w')
            rects = []
            for box in images[key]["regions"]:
                b = box["shape_attributes"]
                x1 = b["x"]/w
                y1 = b["y"]/h
                bw = b["width"]/w
                bh = b["height"]/h
                xc = x1 + (bw/2)
                yc = y1 + (bh/2)
                lbl = box["region_attributes"]["type"]
                if lbl == "1":
                    lbl = "1"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                    rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
                elif lbl == "0":
                    lbl = "0"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            bboxs = bboxs[1:]
            f.write(bboxs)
            f.close()
            img = np.array(img)
            bbs = BoundingBoxesOnImage(rects,shape=img.shape)
            count += 1
            phase_count[phase] += 1

1329 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/1.jpg
pv-test-0000000605
1330 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/10.jpg
pv-test-0000000606
1331 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/13.jpg
pv-test-0000000607
1332 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/17.jpg
pv-test-0000000608
1333 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/25.jpg
pv-test-0000000609
1334 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/26.jpg
pv-test-0000000610
1335 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/30.jpg
pv-test-0000000611
1336 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/32.jpg
pv-test-0000000612
1337 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/33.jpg
pv-test-0000000613
1338 https://raw.githubuserco

In [8]:
print(phase_count["train"])
print(phase_count["test"])

4385
633


In [9]:
labels = os.path.join('pv_ext.json')
phase = "train"
degs = [45,90,135,180,225,270,315]
phase_count = {
    "train": 4385,
    "test": 633
}

with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        leng = len(images[key]["regions"])
        bboxs = ""
        if leng > 0:
            name = "pv-" + phase + "-{:010d}".format(phase_count[phase])
            print("{} {}".format(phase_count[phase],url))
            print(name)
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('images',name+".jpg"))
            f = open(os.path.join('labels',name+".txt"),'w')
            rects = []
            for box in images[key]["regions"]:
                b = box["shape_attributes"]
                x1 = b["x"]/w
                y1 = b["y"]/h
                bw = b["width"]/w
                bh = b["height"]/h
                xc = x1 + (bw/2)
                yc = y1 + (bh/2)
                lbl = box["region_attributes"]["type"]
                if lbl == "1":
                    lbl = "1"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                    rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
                elif lbl == "0":
                    lbl = "0"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            bboxs = bboxs[1:]
            f.write(bboxs)
            f.close()
            img = np.array(img)
            bbs = BoundingBoxesOnImage(rects,shape=img.shape)
            phase_count[phase] += 1
            for deg in degs:
                name = "pv-" + phase + "-{:010d}".format(phase_count[phase])
                image_aug, bbs_aug = iaa.Affine(rotate=deg,mode="reflect")(image=img, bounding_boxes=bbs)
                aug_h, aug_w, aug_c = image_aug.shape
                clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=True).clip_out_of_image()
                bboxs = ""
                for bb in clip_off_bbs.bounding_boxes:
                    x1 = bb.x1
                    y1 = bb.y1
                    x2 = bb.x2
                    y2 = bb.y2
                    bb_w = (x2-x1)/aug_w
                    bb_h = (y2-y1)/aug_h
                    bb_xc = (x1/aug_w) + (bb_w/2)
                    bb_yc = (y1/aug_h) + (bb_h/2)
                    bboxs += "\n"+ str(bb.label) + " " + str(bb_xc) + " " + str(bb_yc) + " " + str(bb_w) + " " + str(bb_h)
                if len(clip_off_bbs.bounding_boxes) > 0:
                    bboxs = bboxs[1:]
                    im_aug = Image.fromarray(image_aug)
                    im_aug.save(os.path.join('images',name+".jpg"))
                    f = open(os.path.join('labels',name+".txt"),'w')
                    f.write(bboxs)
                    f.close()
                    print("{} {}".format(phase_count[phase],deg))
                    phase_count[phase] += 1

4385 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000001.jpg
pv-train-0000004385
4386 45
4387 90
4388 135
4389 180
4390 225
4391 270
4392 315
4393 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000002.jpg
pv-train-0000004393
4394 45
4395 90
4396 135
4397 180
4398 225
4399 270
4400 315
4401 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000003.jpg
pv-train-0000004401
4402 45
4403 90
4404 135
4405 180
4406 225
4407 270
4408 315
4409 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000004.jpg
pv-train-0000004409
4410 45
4411 90
4412 135
4413 180
4414 225
4415 270
4416 315
4417 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000006.jpg
pv-train-0000004417
4418 45
4419 90
4420 135
4421 180
4422 225
4423 270
4424 315
4425 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000007.jpg
pv-train-0000004425
4426 45
4427 90
4428 13

4752 135
4753 180
4754 315
4755 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000051.jpg
pv-train-0000004755
4756 45
4757 90
4758 135
4759 180
4760 225
4761 270
4762 315
4763 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000052.jpg
pv-train-0000004763
4764 45
4765 90
4766 180
4767 225
4768 270
4769 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000053.jpg
pv-train-0000004769
4770 45
4771 90
4772 135
4773 180
4774 225
4775 270
4776 315
4777 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000054.jpg
pv-train-0000004777
4778 45
4779 90
4780 135
4781 180
4782 225
4783 270
4784 315
4785 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000055.jpg
pv-train-0000004785
4786 45
4787 90
4788 135
4789 180
4790 225
4791 270
4792 315
4793 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000056.jpg
pv-train-0000004793
4794 45
4795 9

5070 180
5071 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000104.jpg
pv-train-0000005071
5072 45
5073 90
5074 135
5075 180
5076 225
5077 270
5078 315
5079 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000105.jpg
pv-train-0000005079
5080 45
5081 90
5082 135
5083 180
5084 225
5085 270
5086 315
5087 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000106.jpg
pv-train-0000005087
5088 45
5089 90
5090 135
5091 180
5092 225
5093 270
5094 315
5095 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000107.jpg
pv-train-0000005095
5096 45
5097 90
5098 135
5099 180
5100 225
5101 270
5102 315
5103 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000108.jpg
pv-train-0000005103
5104 45
5105 90
5106 135
5107 225
5108 270
5109 315
5110 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000109.jpg
pv-train-0000005110
5111 45
5112 90
5113 13

5412 45
5413 90
5414 135
5415 180
5416 225
5417 270
5418 315
5419 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000155.jpg
pv-train-0000005419
5420 45
5421 90
5422 135
5423 180
5424 225
5425 270
5426 315
5427 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000156.jpg
pv-train-0000005427
5428 45
5429 90
5430 135
5431 180
5432 225
5433 270
5434 315
5435 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000157.jpg
pv-train-0000005435
5436 45
5437 90
5438 135
5439 180
5440 225
5441 270
5442 315
5443 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000158.jpg
pv-train-0000005443
5444 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000159.jpg
pv-train-0000005444
5445 45
5446 90
5447 135
5448 180
5449 225
5450 270
5451 315
5452 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000160.jpg
pv-train-0000005452
5453 45
5454 90
5455 13

5750 45
5751 90
5752 180
5753 225
5754 270
5755 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000207.jpg
pv-train-0000005755
5756 135
5757 180
5758 315
5759 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000208.jpg
pv-train-0000005759
5760 135
5761 180
5762 315
5763 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000209.jpg
pv-train-0000005763
5764 45
5765 90
5766 135
5767 180
5768 225
5769 270
5770 315
5771 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000210.jpg
pv-train-0000005771
5772 45
5773 90
5774 135
5775 180
5776 225
5777 270
5778 315
5779 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000211.jpg
pv-train-0000005779
5780 45
5781 90
5782 135
5783 180
5784 225
5785 270
5786 315
5787 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000212.jpg
pv-train-0000005787
5788 45
5789 180
5790 225
5791 https://raw.githu

...

8424 270
8425 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000621.jpg
pv-train-0000008425
8426 45
8427 90
8428 135
8429 180
8430 225
8431 270
8432 315
8433 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000622.jpg
pv-train-0000008433
8434 135
8435 180
8436 315
8437 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000623.jpg
pv-train-0000008437
8438 45
8439 90
8440 135
8441 180
8442 225
8443 270
8444 315
8445 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000624.jpg
pv-train-0000008445
8446 45
8447 90
8448 135
8449 180
8450 225
8451 270
8452 315
8453 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000625.jpg
pv-train-0000008453
8454 45
8455 90
8456 135
8457 180
8458 225
8459 270
8460 315
8461 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000626.jpg
pv-train-0000008461
8462 45
8463 90
8464 135
8465 180
8466 225
8467 

8775 45
8776 90
8777 135
8778 180
8779 225
8780 270
8781 315
8782 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000671.jpg
pv-train-0000008782
8783 180
8784 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000672.jpg
pv-train-0000008784
8785 45
8786 90
8787 135
8788 180
8789 225
8790 270
8791 315
8792 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000673.jpg
pv-train-0000008792
8793 45
8794 90
8795 135
8796 180
8797 225
8798 270
8799 315
8800 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000674.jpg
pv-train-0000008800
8801 45
8802 180
8803 225
8804 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000675.jpg
pv-train-0000008804
8805 45
8806 90
8807 135
8808 180
8809 225
8810 270
8811 315
8812 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000676.jpg
pv-train-0000008812
8813 45
8814 90
8815 135
8816 180
8817 225
8818 2

9127 45
9128 90
9129 135
9130 180
9131 225
9132 270
9133 315
9134 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000721.jpg
pv-train-0000009134
9135 45
9136 90
9137 135
9138 180
9139 225
9140 270
9141 315
9142 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000722.jpg
pv-train-0000009142
9143 135
9144 180
9145 315
9146 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000723.jpg
pv-train-0000009146
9147 45
9148 90
9149 135
9150 180
9151 225
9152 270
9153 315
9154 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000724.jpg
pv-train-0000009154
9155 90
9156 135
9157 180
9158 270
9159 315
9160 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000725.jpg
pv-train-0000009160
9161 45
9162 90
9163 135
9164 180
9165 225
9166 270
9167 315
9168 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000726.jpg
pv-train-0000009168
9169 45
9170 

9475 315
9476 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000771.jpg
pv-train-0000009476
9477 135
9478 180
9479 315
9480 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000772.jpg
pv-train-0000009480
9481 180
9482 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000773.jpg
pv-train-0000009482
9483 45
9484 90
9485 135
9486 180
9487 225
9488 270
9489 315
9490 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000774.jpg
pv-train-0000009490
9491 45
9492 90
9493 135
9494 180
9495 225
9496 270
9497 315
9498 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000775.jpg
pv-train-0000009498
9499 45
9500 90
9501 135
9502 180
9503 225
9504 270
9505 315
9506 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000776.jpg
pv-train-0000009506
9507 45
9508 90
9509 135
9510 180
9511 225
9512 270
9513 315
9514 https://raw.githubusercontent.com/

9811 45
9812 90
9813 135
9814 180
9815 225
9816 270
9817 315
9818 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000822.jpg
pv-train-0000009818
9819 45
9820 90
9821 135
9822 180
9823 225
9824 270
9825 315
9826 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000823.jpg
pv-train-0000009826
9827 90
9828 135
9829 180
9830 270
9831 315
9832 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000824.jpg
pv-train-0000009832
9833 45
9834 90
9835 135
9836 180
9837 225
9838 270
9839 315


In [10]:
print(phase_count["train"])
print(phase_count["test"])

9840
633


In [ ]:
# summary test set
files = glob.glob("images/pv-test-*.jpg")
count = 0
count_class = [0,0]

for file in files:
    img = file.split("/")[-1]
    name = img.split('.')[0]
    f = open("labels/"+name+".txt","r")
    lines = list(f)
    f.close()
    print(img)
    for line in lines:
#         print(line)
        class_id, xc, yc, wb, hb = line.split()
        count_class[int(class_id)] += 1
    count += 1
print(count)
print(count_class)

In [ ]:
print(count)
print(count_class)

In [ ]:
# summary train set
files = glob.glob("images/pv-train-*.jpg")
count = 0
count_class = [0,0]

for file in files:
    img = file.split("/")[-1]
    name = img.split('.')[0]
    f = open("labels/"+name+".txt","r")
    lines = list(f)
    f.close()
    print(img)
    for line in lines:
#         print(line)
        class_id, xc, yc, wb, hb = line.split()
        count_class[int(class_id)] += 1
    count += 1
print(count)
print(count_class)

In [ ]:
print(count)
print(count_class)

In [ ]:
# clear empty box
# files = glob.glob("images/pv-train-*.jpg")
# count = 0
# for file in files:
#     img = file.split("/")[-1]
#     name = img.split('.')[0]
#     f = open("labels/"+name+".txt","r")
#     txt = list(f)
#     f.close()
#     if len(txt) == 0:
#         print(img)
#         os.remove("labels/"+name+".txt")
#         os.remove(file)
#         count += 1
# print(count)

In [ ]:
# img = imread('malaria/images/2559636b-f01a-4414-93da-210c3b12d153.png')
# image_aug = iaa.Affine(rotate=45)(image=img)
# imsave('aug01.png',image_aug)